# Multi-agent Customer Support Automation

In this lesson, you will use about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew, LLM

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at IBM and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
)

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at IBM and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.ibm.com/about?lnk=hpii1us")

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.



### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [10]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True,
    memory=True,
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [11]:
inputs = {
    "customer": "Salesforce",
    "person": "Marc Benioff, CEO of Salesforce",
    "inquiry": "I need help with setting up a IBM "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Salesforce just reached out with a super important ask:
I need help with setting up a IBM and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Marc Benioff, CEO of Salesforce from Salesforce is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: I need to gather more information about IBM's resources and guidance relevant to Salesforce's inquiry regarding memory and Crew setup within the IBM framework.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 

About | IBM
Home
About
About IBM
Creators, partners and clients putting technology to work in the real world
Get the latest IBM news
What IBM does
The mission of IBM is to be a catalyst that makes the world work better. We aim to have a positi

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

Hello Marc,

Thank you for reaching out with your important inquiry about setting up an IBM system and adding memory capabilities to your Crew. We genuinely appreciate your interest in enhancing your operational efficiency through advanced technological solutions. Below is a comprehensive guide that covers everything you need to know to successfully integrate memory capabilities into your Crew using IBM technologies:

1. **Understanding Memory Integration**:
   Integrating memory into your Crew allows agents to not only store but also retrieve vital information relevant to their tasks. This enhancement enables your Crew to perform intelligently, adapting to the working environment and the specific needs of users over time.

2. **Role-Based Agents**:
   - Create specialized agents within your Crew that have designated roles focused specifically on memory functions. For instance, you can develop dedicated memory storage and retrieval agents that handle the gathering and processing of information.
   - These agents should possess the ability to learn from their interactions, meaning they can store useful data points and retrieve them later, thereby enhancing their decision-making capabilities.

3. **Utilizing Flexible Tools and APIs**:
   - Equip your memory agents with custom tools and Application Programming Interfaces (APIs) that facilitate effective communication with external databases and services.
   - This setup ensures seamless storage and retrieval of critical information, which is essential for the efficient completion of tasks. The more flexible the tools, the better your agents can operate in real-time scenarios.

4. **Facilitating Intelligent Collaboration**:
   - Encourage your memory agents to share insights and coordinate efficiently with other agents in the Crew. This collaboration not only promotes teamwork but also ensures that stored information is utilized effectively, leading to improved performance and quicker decision-making.
   - By fostering an environment for intelligent collaboration, your Crew will benefit from collective memory retrieval and sharing of knowledge.

5. **Task Management**:
   - Develop structured workflows that clearly outline memory-related tasks, whether they are to be executed sequentially or in parallel. 
   - By carefully managing these tasks, you can ensure that memory capabilities are integrated into your Crew's overall processes without disrupting existing workflows, ultimately enhancing operational efficiency.

6. **Leveraging IBM Technology**:
   - Make use of IBM’s consulting and software solutions that provide adaptive AI services to help you build and customize your memory capabilities. These resources can assist you in modernizing your operations.
   - Additionally, IBM's commitment to sustainable and ethical AI practices ensures that your integration efforts align with long-term organizational goals, providing peace of mind.

By following these strategies, you can effectively add memory capabilities to your Crew, ensuring it can store, access, and utilize information intelligently throughout its operations. I highly recommend consulting IBM documentation and resources for extended tools available for enhancing memory integration.

If you require personalized assistance or have any further questions as you embark on this integration journey, please do not hesitate to reach out to our support team. We are here to help you achieve your goals and optimize the performance of your Crew.

Thank you for choosing IBM, and we look forward to supporting your endeavors in enhancing your Crew's intelligence and performance through advanced memory integration.

Warm regards,

[Your Name]  
Senior Support Representative  
IBM Support Team